In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper

In [2]:
from LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from relation_templates.templates import relations, nominalized_relations

In [3]:
from TypologyQuerier import TypologyQuerier 

In [4]:
logging.set_verbosity_error()

In [5]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]

In [6]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
#relations = nominalized_relations

In [7]:
TOP_K = 1

In [8]:
#unmasker = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking', top_k=100)
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [9]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [10]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)

In [11]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 29395.90it/s]


In [12]:
%%time
unmasker.save_to_cache()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [13]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      9594 |   34039 |    28.1853 |
| compound         |      4452 |   34039 |    13.0791 |
| complex          |      4421 |   34039 |    12.988  |
| compound-complex |      5413 |   34039 |    15.9023 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       610 |     937 |    65.1014 |
| compound         |       604 |     937 |    64.461  |
| complex          |       595 |     937 |    63.5005 |
| compound-complex |       451 |     937 |    48.1323 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |      6689 |   20928 |    31.962  |
| compound         |      3040 |   20928 |    14.526  |
| complex          |      3487 |   20928 |    16.

In [14]:
querier.print_result()

P159: headquarters location N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       313 |     967 |    32.3681 |
| compound         |       334 |     967 |    34.5398 |
| complex          |       329 |     967 |    34.0228 |
| compound-complex |       330 |     967 |    34.1262 |
P37: official language N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       527 |     966 |    54.5549 |
| compound         |       391 |     966 |    40.4762 |
| complex          |       258 |     966 |    26.7081 |
| compound-complex |       372 |     966 |    38.5093 |
P1412: languages spoken, written or signed N:M
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       630 |     969 |   65.0155  |
| compound         |        23 |     969 |    2.37358 

In [15]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 73.93 & 72.65 & 73.93 & 73.50 \\
P36 & capital & 1:1 & 62.16 & 61.74 & 60.03 & 39.69 \\
P103 & native language & N:1 & 72.16 & 76.15 & 76.36 & 69.91 \\
P127 & owned by & N:1 & 34.79 & 0.00 & 1.46 & 25.76 \\
P131 & located in the administrative territorial entity & N:1 & 23.27 & 0.00 & 0.00 & 0.11 \\
P136 & genre & N:1 & 0.64 & 0.00 & 4.19 & 0.00 \\
P138 & named after & N:1 & 3.88 & 8.22 & 3.88 & 12.09 \\
P140 & religion or worldview & N:1 & 75.05 & 47.36 & 74.84 & 37.84 \\
P159 & headquarters location & N:1 & 32.37 & 34.54 & 34.02 & 34.13 \\
P17 & country & N:1 & 31.29 & 3.66 & 35.81 & 15.38 \\
P176 & manufacturer & N:1 & 85.64 & 66.50 & 40.02 & 80.86 \\
P19 & place of birth & N:1 & 21.08 & 0.32 & 0.21 & 0.00 \\
P20 & place of death & N:1 & 27.91 & 0.00 & 0.10 & 0.00 \\
P264 & record label & N:1 & 9.32 & 0.23 & 0.00 & 0.00 \\
P276 & location & N:1 & 41.50 & 39.83 & 33.79 & 41.81 \\
P279 & subclass of & N:1 & 30.60 & 3.42 & 9.96 & 2.07 \\
P30 & continent & N:1

In [16]:
querier.print_for_calculation()

[['relations', 'simple', 'compound', 'complex', 'compound-complex'],
 ['P1001',
  57.63195435092725,
  2.4251069900142657,
  19.828815977175463,
  14.693295292439373],
 ['P101', 9.913793103448276, 2.2988505747126435, 0.0, 0.0],
 ['P103',
  72.1596724667349,
  76.15148413510747,
  76.35619242579324,
  69.9078812691914],
 ['P106', 0.6263048016701461, 0.0, 0.0, 0.0],
 ['P108', 3.3942558746736298, 0.0, 0.0, 0.0],
 ['P127', 34.78893740902475, 0.0, 1.455604075691412, 25.76419213973799],
 ['P1303', 5.690200210748156, 0.0, 0.0, 0.0],
 ['P131', 23.269012485811576, 0.0, 0.0, 0.11350737797956867],
 ['P136', 0.644468313641246, 0.0, 4.189044038668099, 0.0],
 ['P1376',
  73.93162393162393,
  72.64957264957265,
  73.93162393162393,
  73.50427350427351],
 ['P138',
  3.875968992248062,
  8.217054263565892,
  3.875968992248062,
  12.093023255813954],
 ['P140',
  75.05285412262155,
  47.35729386892177,
  74.84143763213531,
  37.84355179704017],
 ['P1412', 65.01547987616098, 2.3735810113519094, 0.0, 9.907

In [17]:
TOP_K = 10

In [18]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [19]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [20]:
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 26214.07it/s]


In [21]:
%%time
unmasker.save_to_cache()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


In [22]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |     19653 |   34039 |    57.7367 |
| compound         |     12777 |   34039 |    37.5364 |
| complex          |     12727 |   34039 |    37.3895 |
| compound-complex |     13329 |   34039 |    39.158  |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       767 |     937 |    81.857  |
| compound         |       762 |     937 |    81.3234 |
| complex          |       757 |     937 |    80.7898 |
| compound-complex |       727 |     937 |    77.588  |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |     13441 |   20928 |    64.225  |
| compound         |      8037 |   20928 |    38.4031 |
| complex          |      9297 |   20928 |    44.

In [23]:
querier.print_result()

P159: headquarters location N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       499 |     967 |    51.6029 |
| compound         |       569 |     967 |    58.8418 |
| complex          |       557 |     967 |    57.6008 |
| compound-complex |       550 |     967 |    56.8769 |
P37: official language N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       879 |     966 |    90.9938 |
| compound         |       841 |     966 |    87.06   |
| complex          |       823 |     966 |    85.1967 |
| compound-complex |       807 |     966 |    83.5404 |
P1412: languages spoken, written or signed N:M
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       872 |     969 |    89.9897 |
| compound         |       837 |     969 |    86.3777 

In [24]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 91.45 & 91.45 & 91.03 & 91.03 \\
P36 & capital & 1:1 & 78.66 & 77.95 & 77.38 & 73.12 \\
P103 & native language & N:1 & 96.42 & 97.85 & 97.34 & 97.54 \\
P127 & owned by & N:1 & 61.86 & 12.52 & 18.92 & 54.88 \\
P131 & located in the administrative territorial entity & N:1 & 54.48 & 16.12 & 21.57 & 10.90 \\
P136 & genre & N:1 & 44.15 & 2.58 & 49.52 & 1.50 \\
P138 & named after & N:1 & 69.46 & 64.34 & 22.95 & 68.37 \\
P140 & religion or worldview & N:1 & 98.73 & 97.04 & 97.46 & 86.05 \\
P159 & headquarters location & N:1 & 51.60 & 58.84 & 57.60 & 56.88 \\
P17 & country & N:1 & 67.53 & 52.69 & 73.55 & 75.38 \\
P176 & manufacturer & N:1 & 94.30 & 90.33 & 70.67 & 90.84 \\
P19 & place of birth & N:1 & 47.67 & 3.60 & 2.22 & 2.44 \\
P20 & place of death & N:1 & 60.86 & 3.15 & 1.78 & 1.57 \\
P264 & record label & N:1 & 31.47 & 20.98 & 2.33 & 20.98 \\
P276 & location & N:1 & 65.59 & 61.52 & 59.54 & 63.30 \\
P279 & subclass of & N:1 & 68.98 & 16.80 & 22.93 & 16.29 \\
P30 

In [25]:
TOP_K = 100

In [26]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [27]:
querier.reset()
querier.set_top_k(TOP_K)
querier.set_model(unmasker)

In [28]:
querier.query(TREx.data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [02:29<00:00,  6.56it/s]


In [29]:
%%time
unmasker.save_to_cache()

CPU times: user 8.4 s, sys: 526 ms, total: 8.92 s
Wall time: 8.94 s


In [30]:
querier.print_global_result()

total Result
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |     29028 |   34039 |    85.2787 |
| compound         |     21002 |   34039 |    61.6998 |
| complex          |     21842 |   34039 |    64.1676 |
| compound-complex |     21027 |   34039 |    61.7733 |
Total Result for 1:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       843 |     937 |    89.968  |
| compound         |       847 |     937 |    90.3949 |
| complex          |       840 |     937 |    89.6478 |
| compound-complex |       831 |     937 |    88.6873 |
Total Result for N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |     18023 |   20928 |    86.1191 |
| compound         |     12916 |   20928 |    61.7164 |
| complex          |     14298 |   20928 |    68.

In [31]:
querier.print_result()

P159: headquarters location N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       765 |     967 |    79.1107 |
| compound         |       813 |     967 |    84.0745 |
| complex          |       810 |     967 |    83.7642 |
| compound-complex |       782 |     967 |    80.8687 |
P37: official language N:1
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       963 |     966 |    99.6894 |
| compound         |       963 |     966 |    99.6894 |
| complex          |       963 |     966 |    99.6894 |
| compound-complex |       959 |     966 |    99.2754 |
P1412: languages spoken, written or signed N:M
| type             |   correct |   total |   accuracy |
|------------------+-----------+---------+------------|
| simple           |       959 |     969 |    98.968  |
| compound         |       953 |     969 |    98.3488 

In [32]:
querier.print_for_latex()

P1376 & capital of & 1:1 & 96.58 & 96.58 & 97.01 & 96.58 \\
P36 & capital & 1:1 & 87.77 & 88.34 & 87.20 & 86.06 \\
P103 & native language & N:1 & 99.90 & 100.00 & 99.90 & 100.00 \\
P127 & owned by & N:1 & 82.82 & 25.62 & 43.67 & 76.27 \\
P131 & located in the administrative territorial entity & N:1 & 83.65 & 52.67 & 69.58 & 47.79 \\
P136 & genre & N:1 & 75.73 & 40.60 & 81.74 & 8.81 \\
P138 & named after & N:1 & 84.81 & 82.95 & 67.75 & 84.03 \\
P140 & religion or worldview & N:1 & 100.00 & 99.79 & 99.37 & 99.15 \\
P159 & headquarters location & N:1 & 79.11 & 84.07 & 83.76 & 80.87 \\
P17 & country & N:1 & 91.18 & 91.40 & 95.05 & 96.24 \\
P176 & manufacturer & N:1 & 98.68 & 97.25 & 82.69 & 96.74 \\
P19 & place of birth & N:1 & 80.51 & 34.11 & 23.31 & 24.47 \\
P20 & place of death & N:1 & 83.53 & 44.28 & 36.10 & 34.84 \\
P264 & record label & N:1 & 78.79 & 66.20 & 31.24 & 61.54 \\
P276 & location & N:1 & 86.24 & 82.59 & 80.50 & 83.32 \\
P279 & subclass of & N:1 & 80.81 & 47.10 & 44.09 & 42